In [200]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [202]:
df = pd.read_csv("Approach to Social Media Cyberbullying and Harassment Detection Using Advanced Machine Learning.csv")
df.head(10)

,Text,Label,Types
0,Ten outside soon doctor shake everyone treatme...,Not-Bullying,NaN
1,my life has come to a standstill and at this p...,Not-Bullying,NaN
2,girl this nigga make me sick to my stomach,Bullying,Ethnicity
3,I wanna fuck you,Bullying,Sexual
4,"Oh hey, you should be ashamed of your disgusti...",Not - Bullying,NaN
5,Haha isn't route running part of the position?...,Bullying,Ethnicity
6,Racism is when u select yur white clothes to w...,Bullying,Ethnicity
7,Who are islamists? Those who fully follow the ...,Bullying,Religion
8,Male action movies...ehhhh @That_Damn_Ram Not ...,Bullying,Religion
9,Wishing my arena partner was on.,Not Bullying,NaN


In [204]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8452 entries, 0 to 8451
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    8452 non-null   object
 1   Label   8450 non-null   object
 2   Types   4825 non-null   object
dtypes: object(3)
memory usage: 198.2+ KB


In [206]:
df.columns

Index(['Text', 'Label', 'Types'], dtype='object')

In [208]:
print("Unique Labels: ")
print(df['Label'].unique())
print()
print("Unique Types: ")
print(df['Types'].unique())

Unique Labels: 
['Not-Bullying' 'Bullying' 'Not - Bullying' 'Bullying ' 'Not Bullying '
 'Not- Bullying ' 'Not -Bullying' 'Not-Bullying ' 'Not -Bullying '
 'Not bullying ' 'Bullying.' 'Not-Bulyying' 'Not-Bulllying' 'Bullyiing'
 'Not-Bulying' 'Not- Bullying' 'Bullyibg' 'Bullying  ' nan 'Bullyinfg']

Unique Types: 
[nan 'Ethnicity' 'Sexual' 'Religion ' 'Religion' 'Threats ' 'Troll'
 'Troll ' 'sexual' 'Vocational' 'Threats' 'Vocational ' 'Political'
 'Ethnicity ' 'Religious ' 'Sexual ' 'Political ' 'Threat' 'Racism'
 'Vocation' 'Saxual' 'political ' 'Religon' 'Ethnically ' 'Threat ']


In [210]:
df[df['Label'].isna()]

,Text,Label,Types
1303,"Wow, stunning photo!""",NaN,NaN
2055,"Your posts always brighten up my feed!""",NaN,NaN


In [212]:
df[df['Types'].isna()]

,Text,Label,Types
0,Ten outside soon doctor shake everyone treatme...,Not-Bullying,NaN
1,my life has come to a standstill and at this p...,Not-Bullying,NaN
4,"Oh hey, you should be ashamed of your disgusti...",Not - Bullying,NaN
9,Wishing my arena partner was on.,Not Bullying,NaN
10,"I hope this round humbled the girls, they can'...",Not- Bullying,NaN
...,...,...,...
8433,"Why does god require you to defend Islam, kill...",Not-Bullying,NaN
8434,I appreciate you revealing your true identity ...,Not-Bullying,NaN
8435,Donald Trump's position on radical Islamic ter...,Not-Bullying,NaN
8436,The terrorists do not speak for Islam; instead...,Not-Bullying,NaN


In [214]:
df.isna().sum()

Text        0
Label       2
Types    3627
dtype: int64

In [216]:
df = df.dropna(subset=["Label"])

In [218]:
df.loc[df["Label"].str.startswith("N"),"Label"] = "NB"
df.loc[df["Label"].str.startswith("B"),"Label"] = "B"
df.head()

,Text,Label,Types
0,Ten outside soon doctor shake everyone treatme...,NB,NaN
1,my life has come to a standstill and at this p...,NB,NaN
2,girl this nigga make me sick to my stomach,B,Ethnicity
3,I wanna fuck you,B,Sexual
4,"Oh hey, you should be ashamed of your disgusti...",NB,NaN


In [220]:
df['Types'] = df[['Types']].fillna(value="Unknown")

In [222]:
df.isna().count()

Text     8450
Label    8450
Types    8450
dtype: int64

In [224]:
df['Types'] = df['Types'].apply(lambda s:s.lower().strip())

In [226]:
d = {"e":"ethnicity","r":"religion","s":"sexual","v":"vocational",'th':"threat",'tr':"troll"}
def map_type(s):
    if s[:2] in d:
        return d[s[:2]]
    elif s[0] in d:
        return d[s[0]]
    else:
        return "unknown"

df['Types'] = df['Types'].apply(map_type)

df["Types"].unique()

array(['unknown', 'ethnicity', 'sexual', 'religion', 'threat', 'troll',
       'vocational'], dtype=object)

In [228]:
df.head()

,Text,Label,Types
0,Ten outside soon doctor shake everyone treatme...,NB,unknown
1,my life has come to a standstill and at this p...,NB,unknown
2,girl this nigga make me sick to my stomach,B,ethnicity
3,I wanna fuck you,B,sexual
4,"Oh hey, you should be ashamed of your disgusti...",NB,unknown


In [230]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)              # remove URLs
    text = re.sub(r"[^a-z\s]", "", text)             # remove punctuation and numbers
    text = re.sub(r"\s+", " ", text).strip()         # remove extra spaces
    return text

df['Text'] = df['Text'].apply(clean_text)

In [232]:
df['Label'] = df['Label'].map({'B': 1, 'NB': 0})

In [234]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import joblib

try:
    from xgboost import XGBClassifier
    xgb_available = True
except ImportError:
    xgb_available = False

In [236]:
X = df[['Text']]
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [245]:
# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(ngram_range=(1,2), max_features=5000), 'Text')
    ]
)

In [247]:
# List of models to try
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "SVM": SVC(kernel='linear')
}

if xgb_available:
    models["XGBoost"] = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Train and evaluate all models
for name, model in models.items():
    print(f"\n🔍 Model: {name}")
    pipeline = Pipeline([
        ('features', preprocessor),
        ('classifier', model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    print(classification_report(y_test, y_pred))


🔍 Model: Logistic Regression
              precision    recall  f1-score   support

           0       0.92      0.82      0.87       723
           1       0.88      0.94      0.91       967

    accuracy                           0.89      1690
   macro avg       0.90      0.88      0.89      1690
weighted avg       0.89      0.89      0.89      1690


🔍 Model: Random Forest
              precision    recall  f1-score   support

           0       0.89      0.85      0.87       723
           1       0.89      0.92      0.91       967

    accuracy                           0.89      1690
   macro avg       0.89      0.89      0.89      1690
weighted avg       0.89      0.89      0.89      1690


🔍 Model: SVM
              precision    recall  f1-score   support

           0       0.90      0.87      0.89       723
           1       0.90      0.93      0.92       967

    accuracy                           0.90      1690
   macro avg       0.90      0.90      0.90      1690
weight

C:\Users\User\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:59:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

           0       0.91      0.79      0.85       723
           1       0.86      0.94      0.90       967

    accuracy                           0.88      1690
   macro avg       0.88      0.87      0.87      1690
weighted avg       0.88      0.88      0.88      1690



In [249]:
# Final SVM pipeline
pipeline = Pipeline([
    ('features', preprocessor),
    ('classifier', SVC(kernel='linear', probability=True))
])

# Train on full dataset
pipeline.fit(X, y)

# Save model
joblib.dump(pipeline, 'svm_hate_speech_model.pkl')
print("Model trained and saved as 'svm_hate_speech_model.pkl'")

Model trained and saved as 'svm_hate_speech_model.pkl'


# I did not want to include the types because in real life comments, there will not be any types mentioned

In [259]:
# Check
model = joblib.load('svm_hate_speech_model.pkl')

In [261]:
# Sample test data
test_data = pd.DataFrame({
    'Text': [
        "I hate people like you",                         # likely hate speech
        "You're amazing and kind!",                      # non-hate
        "You disgusting troll, go away",                 # insult/troll
        "Your religion is nonsense",                     # religion hate
        "We must protect every ethnicity",              # neutral
        "I want to sleep with you tonight",              # sexual
    ],
    'Types': [
        "threat", "unknown", "troll", "religion", "unknown", "sexual"
    ]
})

# Predict
predictions = model.predict(test_data)
proba = model.predict_proba(test_data)

# Add results
test_data["Prediction"] = predictions
test_data["Confidence (Hate)"] = proba[:,1].round(2)
test_data["Label"] = test_data["Prediction"].map({1: "Hate", 0: "Non-Hate"})

print(test_data[["Text", "Types", "Label", "Confidence (Hate)"]])


                               Text     Types     Label  Confidence (Hate)
0            I hate people like you    threat      Hate               0.60
1          You're amazing and kind!   unknown  Non-Hate               0.06
2     You disgusting troll, go away     troll      Hate               0.60
3         Your religion is nonsense  religion      Hate               0.94
4   We must protect every ethnicity   unknown  Non-Hate               0.17
5  I want to sleep with you tonight    sexual      Hate               0.94
